In [6]:
import os
import pickle
import numpy as np
import our_Wolpertinger

from our_Wolpertinger import Wolpertinger
from tqdm import tqdm

import torch

import sys
sys.path.append(os.path.join(os.getcwd(), 'Simulator'))
from WordCounting_Wolpertinger import WordCountingEnv

In [51]:
%load_ext autoreload
%autoreload 2

In [2]:
with open(f"./offline/random_wolpertinger_0.pkl", 'rb') as f:
    buffer = pickle.load(f)

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
buffer.device = device

In [3]:
s = buffer.sample(1)
state_dim = s[0].shape[-1]
action_dim = s[1].shape[-1]

In [42]:
model = Wolpertinger(state_dim, action_dim, n_machines=5, n_exe=20, noise_clip=0.2)

In [67]:
def eval_policy(policy, eval_env, eval_episodes=1):
    avg_reward = 0.
    step_count = 0
    for _ in range(eval_episodes):
        state, done = eval_env.reset(), False
        while not done:
            action = policy.select_action(np.array(state))
            state, reward, done, _ = eval_env.step(action)
            avg_reward += reward
            step_count += 1
            if step_count >= 10:
                break

    avg_reward /= eval_episodes

    print("---------------------------------------")
    print(f"Evaluation over {eval_episodes} episodes: {avg_reward:.3f} steps:{avg_reward/step_count:.3f}")
    print("---------------------------------------")
    return avg_reward/step_count

In [68]:
eval_env = WordCountingEnv(seed=101)
eval_results = []

Building Topology
spout 2
WordCount 9
Database 9


C:\Users\87312\anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [45]:
for i in tqdm(range(20000)):
    model.train(buffer,batch_size=32)
    if i % 10000 == 0 and i > 2000:
        eval_results.append(eval_policy(model, eval_env))

 50%|████▉     | 9993/20000 [00:38<00:38, 257.06it/s]

0.1 collected with offset=-9000.0
0.13 collected with offset=-8700.0


 50%|█████     | 10000/20000 [00:51<00:51, 192.57it/s]


KeyboardInterrupt: 

# BCQ

In [69]:
from bcq import BCQ
offline_model = BCQ(state_dim, action_dim, 1, device)

In [70]:
offline_model.train(buffer, 5000)
print('finish training')
eval_results.append(eval_policy(offline_model, eval_env))